# Data Import, Cleaning, and Preparation

This module is used to query the postgreSQL database in order to retrieve the Yelp and Violations dataset. There will be several steps to do this:
* Import flattened violations dataset. We will use a SQL query to perform aggregation of violation data by restaurant and inspeciton date. 
* Aggregate the Yelp Data
    * Reviews for a given establishment will be aggregated so that reviews *after* the previous inspection (or the earliest review date) and *before* the date of a given inspection are in one batch. 
    * Aggregate "count" features using this same logic
    * Combine the review document for a restaurant into a CLOB using the same logic
* LEFT JOIN the violations dataset to the Yelp data after aggregationis complete for both datasets

In [7]:
#set up connection to our DB
import psycopg2 as psy
import pandas as pd
conn = psy.connect(database="sterndsyelp", 
                        user="mvsternds", 
                        password="nyustern123!", 
                        host="sterndsyelp.cawzspvmqd5q.us-east-1.rds.amazonaws.com", 
                        port="5432"
                       )
#open cursor and check our tables in the DB
cur = conn.cursor()
cur.execute("SELECT table_name FROM information_schema.tables"
            " WHERE table_schema='public'" 
            " AND table_type='BASE TABLE'")
rows =cur.fetchall()
print(pd.DataFrame(rows))

                       0
0               business
1                checkin
2             trnt_insps
3  violations_pittsburgh
4                   tips
5                reviews


This code wil return a dataframe of the Toronto inspections data

In [10]:
cur.execute("SELECT * FROM public.trnt_inspn_agg LIMIT 500")
rows =cur.fetchall()
rows = pd.DataFrame(rows)
rows

,0,1,2,3,4,5,6,7,8
0,10348267,OLLIFFE,1097 YONGE ST,11/30/15,2,0,0,0,0
1,10378852,CORNED BEEF HOUSE,270 ADELAIDE ST W,5/13/15,1,0,0,0,0
2,10377653,BLOOR STREET MARKET,55 BLOOR ST W,1/30/17,1,2,0,0,2
3,10414022,BIRYANI KING,680 REXDALE BLVD,11/17/15,0,0,0,0,0
4,9013311,SAIGON FLOWER RESTAURANT,1138 QUEEN ST W,7/15/15,0,0,0,0,0
5,10567405,NANDO'S QUEEN ST.,739 QUEEN ST W,1/26/17,0,2,0,0,2
6,10277414,THE BIG GUY'S LITTLE COFFEE SHOP,2861 LAKE SHORE BLVD W,5/19/15,0,0,0,0,0
7,10218156,DIAN CHILDCARE CENTRE,283 PHARMACY AVE,7/10/15,0,0,0,0,0
8,10472630,Kairali,1210 KENNEDY RD,8/11/15,2,6,2,0,8
9,10501674,YORK MILLS COLLEGIATE CAFETERIA,490 YORK MILLS RD,12/5/16,0,2,1,0,3
